In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt #used for visualization purpose
import seaborn as sb     #used for visualization purpose
import tensorflow as tf # framework
import PIL
from tensorflow import keras # keras was also a framework not one anymore, it is under tensorflow
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


In [ ]:
from google.colab import drive #adding the file from Google Drive
drive.mount("/content/gdrive")



Mounted at /content/gdrive


In [ ]:
import os
import pathlib

In [ ]:
data_dir=os.path.join("/content/gdrive/MyDrive/Project_Dataset")
data_dir=pathlib.Path(data_dir).with_suffix("")

In [ ]:
#a= os.scandir(data_dir)
#for i in a:
 # print(i.name)

# Pre-Processing

In [ ]:
batch_size=16
img_height=200
img_width=200

In [ ]:
train_ds=tf.keras.utils.image_dataset_from_directory(data_dir,validation_split=0.2,subset="training",seed=123,image_size=(img_height,img_width),batch_size=batch_size)

Found 5619 files belonging to 8 classes.
Using 4496 files for training.


In [ ]:
validation_ds=tf.keras.utils.image_dataset_from_directory(data_dir,validation_split=0.2,subset="validation",seed=123,image_size=(img_height,img_width),batch_size=batch_size)

Found 5619 files belonging to 8 classes.
Using 1123 files for validation.


In [ ]:
class_names=train_ds.class_names
print(class_names)

['CORN-GREY LEAF SPOT', 'POTATO-EARLY BLIGHT', 'POTATO-LATE BLIGHT', 'RICE-BACTERIAL BLIGHT', 'RICE-BROWN SPOT', 'TOMATO-EARLY BLIGHT', 'WHEAT-BROWN RUST', 'WHEAT-YELLOW RUST']


# Pre-processing for all images


In [ ]:
normalization_layer= layers.Rescaling(1./255)

In [ ]:
normalized_ds=train_ds.map(lambda x,y:(normalization_layer(x),y))
image_batch, labels_batch=next(iter(normalized_ds))
first_image=image_batch[0]
print(np.min(first_image),np.max(first_image))

0.0 0.89784545


# CNN Model Structure


In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import Conv2D

In [ ]:
# CNN Model Design
num_classes=len(class_names)
model=Sequential([
    layers.Rescaling(1./255,input_shape=(img_height,img_width,3)),
    layers.Conv2D(16,3,padding="same",activation="relu"),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),
    layers.Conv2D(32,3,padding="same",activation="relu"),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),
    layers.Conv2D(64,3,padding="same",activation="relu"),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),
    layers.Conv2D(128,3,padding="same",activation="relu"),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128,activation="relu"),
    layers.Dense(num_classes)])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.compile(optimizer="adam",loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=["accuracy"])

In [ ]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ rescaling_1 (Rescaling)              │ (None, 200, 200, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 200, 200, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 100, 100, 16)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 100, 100, 16)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 100, 100, 32)        │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 50, 50, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 50, 50, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 50, 50, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 25, 25, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 25, 25, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 25, 25, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 12, 12, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 18432)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       2,359,424 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 8)                   │           1,032 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,457,896 (9.38 MB)

 Trainable params: 2,457,896 (9.38 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
epochs=100
history=model.fit(train_ds,validation_data=validation_ds,epochs=epochs)

Epoch 1/100
281/281 ━━━━━━━━━━━━━━━━━━━━ 1835s 7s/step - accuracy: 0.3967 - loss: 1.5633 - val_accuracy: 0.7925 - val_loss: 0.6611
Epoch 2/100
281/281 ━━━━━━━━━━━━━━━━━━━━ 25s 90ms/step - accuracy: 0.8020 - loss: 0.5354 - val_accuracy: 0.8255 - val_loss: 0.5496
Epoch 3/100
281/281 ━━━━━━━━━━━━━━━━━━━━ 30s 106ms/step - accuracy: 0.8690 - loss: 0.3669 - val_accuracy: 0.8059 - val_loss: 0.6418
Epoch 4/100
281/281 ━━━━━━━━━━━━━━━━━━━━ 25s 87ms/step - accuracy: 0.8948 - loss: 0.2884 - val_accuracy: 0.8798 - val_loss: 0.3323
Epoch 5/100
281/281 ━━━━━━━━━━━━━━━━━━━━ 41s 88ms/step - accuracy: 0.9059 - loss: 0.2511 - val_accuracy: 0.9012 - val_loss: 0.3131
Epoch 6/100
281/281 ━━━━━━━━━━━━━━━━━━━━ 24s 86ms/step - accuracy: 0.9286 - loss: 0.1949 - val_accuracy: 0.8940 - val_loss: 0.2916
Epoch 7/100
281/281 ━━━━━━━━━━━━━━━━━━━━ 41s 86ms/step - accuracy: 0.9387 - loss: 0.1622 - val_accuracy: 0.8958 - val_loss: 0.3026
Epoch 8/100
281/281 ━━━━━━━━━━━━━━━━━━━━ 24s 86ms/step - accuracy: 0.9433 - loss: 

In [ ]:
model.save("/content/gdrive/MyDrive/Project_Dataset/Crop_Disease_Classification_Model1.h5")